In [1]:
# !pip3 install statsmodels
# !pip3 install gmpy2
# !pip3 install cvxpy
# !pip install Mosek
# !pip install ipywidgets



In [2]:
import os
import pathlib
import sys
from tqdm import tqdm

import pandas as pd
import torch
import torchvision
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Subset
import seaborn as sns
import numpy as np

import matplotlib.pyplot as plt
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"device = {device}")

import cp.transformations as cp_t
import cp.graph_transformations as cp_gt
from cp.graph_cp import GraphCP

from scipy.stats import norm

from utils import ModelManager
from utils import standard_l2_norm

# assignments
datasets_folder = "path_datasets"
models_direction = "path_to_model"


import cvxpy as convex


device = cpu
Torch Graph Models are running on cpu
v16


In [3]:
import pickle

pert_radi=0.12
smoothing_sigma = 0.25

def save_pkl(obj, path):
    with open(path, 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)
def load_pkl(path):
    with open(path, 'rb') as f:
        return pickle.load(f)

y_pred, logits, y_true = load_pkl(f'path_to_logits')
y_pred = y_pred.to(device)
logits = logits.to(device)
y_true = y_true.to(device)

In [4]:
randoms = (torch.rand((1000,)) * (0.73 - 0.72)) + 0.72
sigma = 0.01

def np_upperbound(randoms, SIGMA, radi, alpha=0.05, n_classes=1):
    bon_alpha = alpha / n_classes
    error = 0
    p_upper = torch.minimum(randoms.mean() + error, torch.tensor(1.0).to(randoms.device))
    result = norm.cdf(
        norm.ppf(p_upper.cpu(), scale=SIGMA) + radi,
        scale=SIGMA)
    return torch.tensor(result)
print("NP bound: ", np_upperbound(randoms, pert_radi, sigma))

def dkw_upperbound(randoms, SIGMA, radi, alpha=0.05, num_s=1000, n_classes=1, evasion=True):
    bon_alpha = alpha / n_classes
    error = 0
    s_min = 0
    s_max = 1
    s_seg = torch.linspace(s_min, s_max, num_s)

    empi_cdf = torch.minimum(
        ((randoms.view(-1, 1) > s_seg.to(randoms.device)).sum(dim=0) / randoms.shape[0]) + error,
        torch.tensor([1.0]).to(randoms.device))

    result = (norm.cdf(norm.ppf(empi_cdf.cpu(), scale=SIGMA) + radi, scale=SIGMA) * (1 / (num_s))).sum()
    return torch.tensor(result)
print("DKW bound: ", dkw_upperbound(randoms, pert_radi, sigma))


def dkw_lowerbound(randoms, SIGMA, radi, alpha=0.05, num_s=1000, n_classes=1, evasion=True):
    bon_alpha = alpha / n_classes
    error = 0
    s_min = 0
    s_max = 1
    s_seg = torch.linspace(s_min, s_max, num_s)

    empi_cdf = torch.maximum(
        ((randoms.view(-1, 1) > s_seg.to(randoms.device)).sum(dim=0) / randoms.shape[0]) - error,
        torch.tensor([0.0]).to(randoms.device))

    result = (norm.cdf(norm.ppf(empi_cdf.cpu(), scale=SIGMA) - radi, scale=SIGMA) * (1 / (num_s))).sum()
    return torch.tensor(result)


def np_upperbound_tensor(scores_samplings, SIGMA, radi, alpha=0.05, n_classes=1):
    bon_alpha = alpha / n_classes
    error = 0
    p_uppers = torch.minimum(scores_samplings.mean(dim=-1) + error, torch.tensor(1.0).to(scores_samplings.device))
    result = norm.cdf(
        norm.ppf(p_uppers.cpu(), scale=SIGMA) + radi,
        scale=SIGMA)
    return torch.tensor(result).to(scores_samplings.device)

def dkw_upperbound_tensor(scores_sampling, SIGMA, radi, alpha=0.05, num_s=10000, n_classes=1):
    return torch.stack([
        torch.stack([
            dkw_upperbound(scores_sampling[d, c, :], SIGMA=SIGMA, radi=radi, alpha=alpha, num_s=num_s, n_classes=n_classes)
            for c in range(scores_sampling.shape[1])
        ]) 
        for d in range(scores_sampling.shape[0])
    ]).to(scores_sampling.device)

def dkw_lowerbound_tensor(scores_sampling, SIGMA, radi, alpha=0.05, num_s=10000, n_classes=1):
    return torch.stack([
        torch.stack([
            dkw_lowerbound(scores_sampling[d, c, :], SIGMA=SIGMA, radi=radi, alpha=alpha, num_s=num_s, n_classes=n_classes)
            for c in range(scores_sampling.shape[1])
        ]) 
        for d in range(scores_sampling.shape[0])
    ]).to(scores_sampling.device)


def np_bounds_tensor(scores_samplings, SIGMA, radi, alpha=0.05, n_classes=1):
    bon_alpha = alpha / n_classes
    error = np.sqrt(np.log(1 / bon_alpha) / (2 * scores_samplings.shape[-1]))
    p_uppers = torch.minimum(scores_samplings.mean(dim=-1) + error, torch.tensor(1.0).to(scores_samplings.device))
    p_lowers = torch.maximum(scores_samplings.mean(dim=-1) - error, torch.tensor(0.0).to(scores_samplings.device))

    upper_result = norm.cdf(
        norm.ppf(p_uppers.cpu(), scale=SIGMA) + radi,
        scale=SIGMA)
    lower_result = norm.cdf(
        norm.ppf(p_lowers.cpu(), scale=SIGMA) - radi,
        scale=SIGMA)
    
    return torch.tensor(lower_result).to(scores_samplings.device), torch.tensor(upper_result).to(scores_samplings.device)

NP bound:  tensor(0.7885, dtype=torch.float64)
DKW bound:  tensor(0.7378, dtype=torch.float64)


In [7]:
def get_cal_mask(vals_tensor, fraction=0.1):
    perm = torch.randperm(vals_tensor.shape[0])
    mask = torch.zeros((vals_tensor.shape[0]), dtype=bool)
    cutoff_index = int(vals_tensor.shape[0] * fraction)
    mask[perm[:cutoff_index]] = True
    return mask

def singleton_hit(pred_set, y_true):
    return ((pred_set[y_true])[pred_set.sum(axis=1) == 1].sum() / (pred_set).shape[0]).item()

In [10]:
def find_lpoisoning_quantile(scores, true_cal_mask, coverage=0.9, k=2):

    calibration_labels = true_cal_mask.nonzero(as_tuple=True)[1].cpu().numpy()

    n = scores.shape[0]
    alpha = 1 - coverage
    M = 10  # large positive number (should be as small as possible)
    q = convex.Variable()
    z = convex.Variable(n, boolean=True)
    b = convex.Variable(scores.shape, boolean=True)

    y = convex.sum(convex.multiply(scores, b), 1)

    constraints = [
        convex.sum(z) >= convex.floor(alpha * n),
        convex.sum(1 - z) >= (1 - alpha) * n,
        y <= q + M * (1 - z),
        y >= q - M * z,
        convex.sum(b, 1) == 1, # at most one label per node
        convex.sum(1-b[np.arange(n), calibration_labels]) <= k # at most k changed
    ]

    prob = convex.Problem(convex.Minimize(q), constraints).solve()
    return prob


In [11]:
smoothing_sigma = 0.25

y_true_mask = F.one_hot(y_true).bool()
cp = GraphCP(transformation_sequence=[cp_t.APSTransformation(softmax=True)], coverage_guarantee=0.9)
sc_scores = torch.stack([cp.get_scores_from_logits(logits[:, i, :]) for i in range(logits.shape[1])]).permute(1, 2, 0) + 1
esc_scores = sc_scores.mean(axis=2)
esc_scores.shape
esc_scores = esc_scores.to('cpu')

In [ ]:
coverages = np.array([0.9])
result = []
for budget in [1,2,3]:
    for coverage in coverages:
        print(coverage, budget)
        for iter_i in tqdm(range(10)):
            cal_mask = get_cal_mask(esc_scores, 0.1)
            true_cal_mask = y_true_mask[cal_mask]
            eval_mask = ~cal_mask

            cons_q = find_lpoisoning_quantile(esc_scores[cal_mask], true_cal_mask, coverage=coverage, k=budget)
            pred_set = (esc_scores >= cons_q)

            result.append({
            "iter_i": iter_i,
            "cons_q": cons_q,
            "empi_coverage": cp.coverage(pred_set[eval_mask], y_true[eval_mask]),
            "set_size": cp.average_set_size(pred_set[eval_mask]),
            "singleton_hit": singleton_hit(pred_set[eval_mask], y_true[eval_mask]),
            "1-\\alpha": coverage,
            "k": budget
        })


0.9 1


100%|█████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [08:01<00:00, 48.16s/it]


0.9 2


 50%|██████████████████████████████████████████████▌                                              | 5/10 [29:16<30:50, 370.02s/it]